# JSON mini-project

We will be working with the following json file: ```'world_bank_projects.json'```

## Imports for Python, Pandas

In [1]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import operator

## Import data

In [2]:
# Load json as a string
json_string = json.load((open('data/world_bank_projects.json')))

# Load json as a panda dataframe
json_df = pd.read_json('data/world_bank_projects.json')

## Quick exploration of the data

In [3]:
# Check structure of json_df
json_df.head() 

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,{'$oid': '52b213b38594d8a2be17c782'},2014,November,2013-11-01T00:00:00Z,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,...,TI,IBRD,Active,Y,"{'Percent': 46, 'Name': 'Regional integration'}","[{'code': '47', 'name': 'Regional integration'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en
3,{'$oid': '52b213b38594d8a2be17c783'},2014,October,2013-10-31T00:00:00Z,MIN. OF PLANNING AND INT'L COOPERATION,NaN,Republic of Yemen!$!RY,RY,Republic of Yemen,"Yemen, Republic of",...,JB,IBRD,Active,N,"{'Percent': 50, 'Name': 'Participation and civ...","[{'code': '57', 'name': 'Participation and civ...","59,57",0,1500000,http://www.worldbank.org/projects/P144665?lang=en
4,{'$oid': '52b213b38594d8a2be17c784'},2014,October,2013-10-31T00:00:00Z,MINISTRY OF FINANCE,2019-04-30T00:00:00Z,Kingdom of Lesotho!$!LS,LS,Kingdom of Lesotho,Lesotho,...,"FH,YW,YZ",IBRD,Active,N,"{'Percent': 30, 'Name': 'Export development an...","[{'code': '45', 'name': 'Export development an...","41,45",13100000,13100000,http://www.worldbank.org/projects/P144933/seco...


In [5]:
json_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 50 columns):
_id                         500 non-null object
approvalfy                  500 non-null int64
board_approval_month        500 non-null object
boardapprovaldate           500 non-null object
borrower                    485 non-null object
closingdate                 370 non-null object
country_namecode            500 non-null object
countrycode                 500 non-null object
countryname                 500 non-null object
countryshortname            500 non-null object
docty                       446 non-null object
envassesmentcategorycode    430 non-null object
grantamt                    500 non-null int64
ibrdcommamt                 500 non-null int64
id                          500 non-null object
idacommamt                  500 non-null int64
impagency                   472 non-null object
lendinginstr                495 non-null object
lendinginstrtype            495 non

**Observations:**

- Each row of the dataframe ```json_df``` represents a project.
- Some of the columns have missing values.

## Question 1. 

*Find the 10 countries with most projects.*

**Approach:**
- The column ```countryshortname``` has no missing values.
- Use ```value_counts()``` on ```countryshortname``` to get the number of projects per country. 
- Sort the resulting Panda series.

In [8]:
# Count the number of occurrences for each country 
projects_per_country = json_df.countryshortname.value_counts()

projects_per_country.head(10)

Indonesia             19
China                 19
Vietnam               17
India                 16
Yemen, Republic of    13
Nepal                 12
Morocco               12
Bangladesh            12
Africa                11
Mozambique            11
Name: countryshortname, dtype: int64

**Problem(s):**
- The resulting series containts an entry ```Africa```, which is a continent, not a country. 

**Solution(s):**
- We can clean up ```json_df``` from entries that do NOT correspond to countries.
- ```countrycodes``` for entries other than countries are composed of a number and a letter: 

In [15]:
# Inspect countrycode corresponding to the CONTINENT 'Africa'
json_df[json_df['countryshortname']=='Africa'][['countryshortname','countrycode']].head(1)

,countryshortname,countrycode
45,Africa,3A


- While ```countrycodes``` for countries are composed of two letters:

In [17]:
# Inspect countrycode corresponding to the COUNTRY 'Ethiopia'
json_df[json_df['countryshortname']=='Ethiopia'][['countryshortname','countrycode']].head(1)

,countryshortname,countrycode
0,Ethiopia,ET


- Filter rows that correspond only to countries using the following pattern: '\D{2}' (for two consecutive character matching any non-digit character).

In [19]:
jsondf_countries_only = json_df[json_df['countrycode'].str.match('\D{2}')]
projects_per_country_only = jsondf_countries_only.countryshortname.value_counts()

projects_per_country_only.head(10)

China                 19
Indonesia             19
Vietnam               17
India                 16
Yemen, Republic of    13
Bangladesh            12
Nepal                 12
Morocco               12
Mozambique            11
Burkina Faso           9
Name: countryshortname, dtype: int64

**Note:** 
- There are other countries with as many projects as Brazil.

## Question 2. 

*Find the top 10 major project themes (using column 'mjtheme_namecode')*

In [24]:
# Inspect contents of mjtheme_namecode
json_df.mjtheme_namecode[0]

[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]

In [23]:
json_df.mjtheme_namecode[100]

[{'code': '11', 'name': 'Environment and natural resources management'},
 {'code': '10', 'name': 'Rural development'},
 {'code': '10', 'name': 'Rural development'},
 {'code': '10', 'name': 'Rural development'}]

**Observations:**
- Each project (row) of ```mjtheme_namecode``` contains a list of dictionnaries. 
- Each dictionnary contains a pair of keys ```code``` and ```name``` that are linked.
- Some of the keys ```'name'``` have no value.
- Some lists have several times the same dictionnary.

**Approach:**

- Create a normalized dataframe from json nested string ```mjtheme_namecode``` and ```id``` using project ```id``` to identify different projects. 
- Remove duplicate themes for each project.
- Count the number of occurrences for each code. 

In [26]:
# Normalized dataframe from json_string
# Column 'name' contains blanks so we drop it to further drop duplicates only on counts
json_normalized = json_normalize(json_string, 'mjtheme_namecode',['id']).drop('name',axis=1).drop_duplicates()  

json_normalized['code'].value_counts()

11    157
10    148
2     140
8     128
4     119
6     116
7     114
5      61
9      40
1      33
3      14
Name: code, dtype: int64

## Question 3.

*In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.*

**Approach:**

- Create a normalized dataframe from json nested string ```mjtheme_namecode```.
- Sort values by column ```code``` and ```name``` and replace the blanks in column ```name``` by ```NaN``` values.
- Backfill ```Nan``` values with the next non ```NaN``` value in column ```name```.

In [31]:
# Normalized dataframe from nested string in 'mjtheme_namecode'
mj_themes_namecodes = json_normalize(json_string, 'mjtheme_namecode')
mj_themes_namecodes.head(10)

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
7,6,Social protection and risk management
8,7,Social dev/gender/inclusion
9,7,Social dev/gender/inclusion


In [32]:
# Sort the dataframe by code and names
# This way we can sefely use the bfill method to fill blanks with further 'names'
sorted_mj_theme = mj_themes_namecodes.sort_values(['code','name'])
sorted_mj_theme.head(10)

,code,name
212,1,
363,1,
1024,1,
1114,1,
1437,1,
2,1,Economic management
88,1,Economic management
175,1,Economic management
204,1,Economic management
205,1,Economic management


In [34]:
# Replace blanks in 'name' by NaN for further filling
mj_theme_nan = sorted_mj_theme.replace('', np.nan)
mj_theme_nan.head(10)

,code,name
212,1,NaN
363,1,NaN
1024,1,NaN
1114,1,NaN
1437,1,NaN
2,1,Economic management
88,1,Economic management
175,1,Economic management
204,1,Economic management
205,1,Economic management


In [35]:
# Fill blank names with a bfill method
filled_mj_theme = mj_theme_nan.bfill()
filled_mj_theme.head(10)

,code,name
212,1,Economic management
363,1,Economic management
1024,1,Economic management
1114,1,Economic management
1437,1,Economic management
2,1,Economic management
88,1,Economic management
175,1,Economic management
204,1,Economic management
205,1,Economic management


In [37]:
# Sort indices back to original dataframe
sorted_filled_mj_theme = filled_mj_theme.sort_index()
sorted_filled_mj_theme.head(10)

,code,name
0,8,Human development
1,11,Environment and natural resources management
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
7,6,Social protection and risk management
8,7,Social dev/gender/inclusion
9,7,Social dev/gender/inclusion
